📢 [프롬프트 엔지니어링, LLM(ChatGPT) 기반의 AI 서비스 개발](https://fastcampus.co.kr/data_red_golbin) 강의에서 사용하는 예제입니다.

### 데이터 생성 프롬프트

```
첨부한 기고문에서 질문과 답변 세트를 만들어줘

--- 기고문 ---
{article}
--- 기고문 끝 ---

질문과 답변 세트를 5개 만들어줘.
각 질문과 답변은 전부 다른 내용을 담고 있어야 해.
숫자로 반복문을 돌리지 말고, 첨부한 파일의 내용에 있는 내용을 구체적으로 담아야 해.

질문 답변 필드명은 instruction과 output으로 작성해줘
질문과 답변은 한국어로 작성해줘

출력 형식은 CSV로 출력해줘. 각 항목은 쌍따옴표로 감싸줘.
```

생성한 파일

- CSV 데이터 파일: [the-age-of-AI-has-begun.csv](https://drive.google.com/file/d/13G40JFsrSJB8GgZI4AiC2s0yF3tmVQCj/view?usp=sharing)
- 기고문 텍스트 파일: [The Age of AI has begun](https://docs.google.com/document/d/1nXYbAJRYzQ9To1pXscFZ10YU_5G2A1N-ml4_kbaGIJY/edit?usp=sharing)

In [1]:
!nvidia-smi

Mon Oct  2 04:41:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 모델 로딩

In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

## 데이터셋 로딩

In [7]:
from datasets import load_dataset

data = load_dataset("csv",
                    data_files={"train":"중소기업은행_금융용어 해설_20201231.csv"})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
[row for row in data['train']]

[{'구 분': 1,
  '용 어': '가입기간',
  '설 명': '퇴직연금 설정 이후 당해 사업장에서 근로를 제공한 기간, 당해 퇴직연금의 설정 전에 제공한 근로기간에 대해서도 가입기간으로 할 수 있음'},
 {'구 분': 2, '용 어': '가입연령', '설 명': '퇴직연금에 가입 할 수 있는 최소연령'},
 {'구 분': 3,
  '용 어': '가입자',
  '설 명': '퇴직연금에 가입하여 연금규약에 따라 퇴직급여 수급권을 취득한 근로자 또는 개인퇴직계좌(IRP)를 설정하여 그 계약에 따라 퇴직급여 수급권을 취득한 자'},
 {'구 분': 4,
  '용 어': '가입자 자격상실',
  '설 명': '퇴직연금 가입자가 정년, 중도사망, 중도퇴직 등의 사유로 가입자로서의 자격을 상실하는 것을 말함'},
 {'구 분': 5,
  '용 어': '가입자명부',
  '설 명': '퇴직연금에 가입한 사업장에 속해있는 근로자의 퇴직금정보 등을 포함한 기본사항을 적은 명부'},
 {'구 분': 6,
  '용 어': '개인퇴직계좌 (IRP)',
  '설 명': '퇴직급여제도의 일시금을 수령한 자 등이 그 수령액을 적립 운용하기 위하여 퇴직연금사업자에게 설정한 계좌'},
 {'구 분': 7,
  '용 어': '거치연금',
  '설 명': '연금급여지급 요건을 충족하였으나 급여의 일정 부분이 그 이후에 지급되는 연금'},
 {'구 분': 8,
  '용 어': '거치연금이자율',
  '설 명': '연금자산을 일정기간 동안 거치 후 지급하는 연금제도에서 퇴직 시부터 연금지급 직전까지의 기간 동안 부리해주는 이자율'},
 {'구 분': 9,
  '용 어': '경험승급률(Experienced salary scale)',
  '설 명': '예정승급률의 종류로서 회사로부터 받은 근로자 데이터의 생년월일, 입사일, 기준급여 등을 활용, 연령별 인원 및 기준급여를 집계하여 연령별 기준급여를 추정한 후, 기준연령의 추정급여로 나누어 산출한 연령별 급여지수. 계산시점 현재의 근로자의 급여

In [10]:
data = data.map(
    lambda x: {
        'text': f"""User: {x['용 어']}

Assistant: {x['설 명']}<|endoftext|>""" }
)

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

In [11]:
[row['text'] for row in data['train']]

['User: 가입기간\n\nAssistant: 퇴직연금 설정 이후 당해 사업장에서 근로를 제공한 기간, 당해 퇴직연금의 설정 전에 제공한 근로기간에 대해서도 가입기간으로 할 수 있음<|endoftext|>',
 'User: 가입연령\n\nAssistant: 퇴직연금에 가입 할 수 있는 최소연령<|endoftext|>',
 'User: 가입자\n\nAssistant: 퇴직연금에 가입하여 연금규약에 따라 퇴직급여 수급권을 취득한 근로자 또는 개인퇴직계좌(IRP)를 설정하여 그 계약에 따라 퇴직급여 수급권을 취득한 자<|endoftext|>',
 'User: 가입자 자격상실\n\nAssistant: 퇴직연금 가입자가 정년, 중도사망, 중도퇴직 등의 사유로 가입자로서의 자격을 상실하는 것을 말함<|endoftext|>',
 'User: 가입자명부\n\nAssistant: 퇴직연금에 가입한 사업장에 속해있는 근로자의 퇴직금정보 등을 포함한 기본사항을 적은 명부<|endoftext|>',
 'User: 개인퇴직계좌 (IRP)\n\nAssistant: 퇴직급여제도의 일시금을 수령한 자 등이 그 수령액을 적립 운용하기 위하여 퇴직연금사업자에게 설정한 계좌<|endoftext|>',
 'User: 거치연금\n\nAssistant: 연금급여지급 요건을 충족하였으나 급여의 일정 부분이 그 이후에 지급되는 연금<|endoftext|>',
 'User: 거치연금이자율\n\nAssistant: 연금자산을 일정기간 동안 거치 후 지급하는 연금제도에서 퇴직 시부터 연금지급 직전까지의 기간 동안 부리해주는 이자율<|endoftext|>',
 'User: 경험승급률(Experienced salary scale)\n\nAssistant: 예정승급률의 종류로서 회사로부터 받은 근로자 데이터의 생년월일, 입사일, 기준급여 등을 활용, 연령별 인원 및 기준급여를 집계하여 연령별 기준급여를 추정한 후, 기준연령의 추정급여로 나누어 산출한 연령별 급여지수. 계산시점 현재의 근로자의 급여를 기초로 통계

## 텍스트 데이터 tokenize

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


Map:   0%|          | 0/196 [00:00<?, ? examples/s]

In [13]:
data["train"][0]["text"]

'User: 가입기간\n\nAssistant: 퇴직연금 설정 이후 당해 사업장에서 근로를 제공한 기간, 당해 퇴직연금의 설정 전에 제공한 근로기간에 대해서도 가입기간으로 할 수 있음<|endoftext|>'

In [14]:
tokenized_text = tokenizer(data["train"][0]["text"])
tokenized_text

{'input_ids': [56, 426, 85, 29, 2389, 2645, 202, 202, 36, 12732, 15097, 15741, 29, 5883, 3488, 4203, 1071, 11364, 6364, 363, 2816, 301, 1361, 288, 1725, 15, 11364, 5883, 3488, 285, 4203, 390, 274, 1361, 288, 2816, 2645, 274, 1626, 309, 2389, 2645, 339, 823, 365, 327, 639, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
token_text = [tokenizer.decode(id) for id in tokenized_text["input_ids"]]
", ".join(token_text)

'U, se, r, :,  가입, 기간, \n, \n, A, ss, ist, ant, :,  퇴직, 연금,  설정,  이후,  당해,  사업장, 에서,  근로, 를,  제공, 한,  기간, ,,  당해,  퇴직, 연금, 의,  설정,  전, 에,  제공, 한,  근로, 기간, 에,  대해서, 도,  가입, 기간, 으로,  할,  수,  있, 음, <|endoftext|>'

## PEFT 학습 준비

In [16]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [17]:
# 학습되는 파라미터 수 출력
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"""trainable params: {trainable_params}
all params: {all_param}
trainable%: {100 * trainable_params / all_param}""")

trainable params: 6553600
all params: 6608701440
trainable%: 0.09916622894073424


## 학습하기

In [18]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        max_steps=150, # 100 step 학습시, V100 기준 약 2분 소요
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        learning_rate=1e-4,
        fp16=True,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        logging_steps=25,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,3.269500
50,2.745300
75,2.425800
100,2.248000
125,1.994700
150,2.032300


TrainOutput(global_step=150, training_loss=2.45259760538737, metrics={'train_runtime': 157.43, 'train_samples_per_second': 1.906, 'train_steps_per_second': 0.953, 'total_flos': 1458108328304640.0, 'train_loss': 2.45259760538737, 'epoch': 1.53})

In [19]:
model.eval()
model.config.use_cache = True

In [20]:
from transformers.generation import GenerationConfig

gen_cfg = GenerationConfig.from_model_config(model.config)
gen_cfg.pad_token_id = tokenizer.eos_token_id

def gen(x):
    gen_cfg.temperature = 0.1
    gen_cfg.max_new_tokens = 128

    gened = model.generate(
        **tokenizer(
            f"User: {x}\n\nAssistant:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        do_sample=True,
        generation_config=gen_cfg
    )

    return tokenizer.decode(gened[0])

In [22]:
gen('CD')

'User: CD\n\nAssistant: 퇴직연금제도에서 근로자가 퇴직 후에 받을 퇴직급여를 금융기관에 적립하여 운용한 후 근로자가 퇴직할 때 일시금 또는 연금으로 지급하는 제도. 퇴직연금제도는 확정급여형(DB)과 확정기여형(DC)으로 구분됨. 퇴직연금제도는 근로자의 수급권 보장을 위해 퇴직연금사업자(금융기관)가 매년 1회 이상 정기적으로 부담금을 납입하고, 근로자가 퇴직할 때 연금 또는 일시금으로 지급하는 제도. 퇴직연금제도는 확정급여형(DB)과 확정기여형(DC)으로 구분됨. 퇴직연금제도는'

endoftext 이 안나왔는데 나오려면 1000개 이상의 데이터셋을 학습시켜야 함

## 모델 저장 & 업로드

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
model.push_to_hub('계정명/koalpaca-polyglot-12.8b-bill')

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/golbin/koalpaca-polyglot-12.8b-bill/commit/b5591d3d4fda960320dfa2fa26da2676120d2207', commit_message='Upload model', commit_description='', oid='b5591d3d4fda960320dfa2fa26da2676120d2207', pr_url=None, pr_revision=None, pr_num=None)

Reference: [KoAlpaca QLoRA Training Original Notebook by Beomi](https://colab.research.google.com/gist/Beomi/a3032e4eaa33b86fdf8de1f47f15a647/2023_05_26_bnb_4bit_koalpaca_v1_1a_on_polyglot_ko_12_8b.ipynb)